## Diffuse He-3 in the grain
<br>
  
#### Syntax
`conc0 = ProdDiffHe3Fxn(conc0, HeP0, EDT, HeliumPars)` <br>
#### Input 
`conc0` : concentration prior to diffusion (at g-1) <br>
`HeP0` : production rate of He-3 (spallation); (at g-1 yr-1) <br>
`EDT` : effective diffusion temperature (K) <br>
`HeliumPars` : dictionary of parameters relevant for Helium production/decay <br>
#### Variables Used
##### Helium Pars
`x` : vector of He grain nodes
`R` : R constant
`Ea` : activation energy
`lnD0aa` 
`nx` : number of nodes in the grain
`dx` : grain node spacing
`r` : grain radius (cm) 
#### Output
##### Calc Pars
`He Surf Matrix` : matrix size Hez with concentration of 3He at depth for given grain size and density <br>
#### Notes
Adapted from code sent from Marissa Tremblay for He3 diffusion in a quartz grain.

**Date of Creation:** 7. Juli 2021 <br>
**Author:** Donovan Dennis (after Tremblay) <br>
**Update:** <br>

In [1]:
def ProdDiffHe3Fxn(conc0, HeP0, EDT, HeliumPars):    
    x = HeliumPars['x']
    R = HeliumPars['R']
    Ea = HeliumPars['He_Ea']
    lnD0aa = HeliumPars['lnD0aa']
    a = HeliumPars['a']
    nx = HeliumPars['nx']
    dx = HeliumPars['dx']
    r = HeliumPars['r']
    
    dt = 1
    cminm = 100
    siy = 60 * 60 * 24 * 365.25
    D0 = 1.22e-2 * cminm # Schuster and Farley via Baxter 2010
    D = D0 / r**2 * siy * np.exp(-Ea / R/ EDT)   
    
    beta = (2.0 * dx**2) / (D * dt)
    
    A1 = np.multiply(np.diag(np.ones(nx)), (-2 -beta)) + np.diag(np.ones(nx-1), k = 1) + np.diag(np.ones(nx-1), k = -1)
    A2 = np.multiply(np.diag(np.ones(nx)), (2 - beta)) - np.diag(np.ones(nx-1), k = 1) - np.diag(np.ones(nx-1), k = -1)
    A1[0,0] = A1[0,0] - 1
    A2[0,0] = A2[0,0] + 1 

    unew = np.multiply(conc0, x)
    He_3_prod = np.multiply(HeP0,x) * beta * dt

    b = A2 @ unew - He_3_prod

    unew = np.linalg.solve(A1,b)
    conc0 = np.divide(unew, x)
  
    return conc0